### GenreBot : Using Natural Language Processing in Music Marketing

For background on this project, please see the [README](../README.md).

**Notebooks**
- [Data Acquisition](./01_data_acquisition.ipynb)
- Data Cleaning (this notebook)
- [Exploratory Data Analysis](./03_eda.ipynb)
- [Modeling](./04_modeling.ipynb)
- [Experiments](./04a_experiments.ipynb)
- [Results and Recommendations](./05_results.ipynb)

**In this notebook, you'll find:**
- Analysis and handling of null values in our raw posts dataset
- Regular expressions and other text transformations to make our columns more useful

In [74]:
# usual imports, plus re for regular expressions

import pandas as pd
import re

# https://stackoverflow.com/questions/2087370/decode-html-entities-in-python-string
# need this to transform HTML entities into their actual character strings easily
from w3lib.html import replace_entities

**Assumptions/parameters for cleaning**:
- We'll use the concatenated **title** and **selftext** for our training material.
- Even if the **selftext** of a post has **[deleted]** or **[removed]** as its content, it still potentially contains useful information in its **title**.
- Within our text, we will get rid of:
    - Unicode characters and emojis
    - HTML entities
    - literal escape characters such as **\n** or **\t**
    - URLs
    - Punctuation other than apostrophes
    - Numbers
    - Double spaces within the text, and unnecessary leading/trailing spaces
- **NOTE:** The items above regarding punctuation and numbers could definitely be controversial. Given more time, I would like to reconsider the decision to strip these and find a more sophisticated cleaning strategy.
- We will keep capitalization for now and let the vectorizers deal with lower-casing, so that we can get accurate trending terms at the end.

In [75]:
# Read in our raw posts from notebook 01

df = pd.read_csv('../data/raw_posts.csv')

In [76]:
# Take a look at some useful info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39956 entries, 0 to 39955
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        39956 non-null  object
 1   created_utc   39956 non-null  int64 
 2   id            39956 non-null  object
 3   num_comments  39956 non-null  int64 
 4   score         39956 non-null  int64 
 5   selftext      37647 non-null  object
 6   subreddit     39956 non-null  object
 7   title         39956 non-null  object
 8   period        39956 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 2.7+ MB


**Conclusions**:
- The number of rows looks good - we want around 40,000.
- The only actual nulls we have are in the **selftext** column.
- None of the data types seem problematic right now - most of these we're just going to cart along to EDA anyway!

In [77]:
# Since we want to keep at least the title for posts that have no useful selftext,
# let's fill NaNs with empty string in preparation for concatenation (we don't want to concatenate with NaN!)
df.fillna(value = '', inplace = True)

In [78]:
# Also - we know we saw some [deleted] and [removed] in there - let's see if they at least have titles

df[(df['selftext'] == '[deleted]') | (df['selftext'] == '[removed]')]

,author,created_utc,id,num_comments,score,selftext,subreddit,title,period
2,[deleted],1223963408,76zmj,2,2,[deleted],Jazz,anybody have that record of Charlie Christian ...,1
6,[deleted],1244717722,8rmly,2,2,[deleted],Jazz,What happened to urge2burge.wordpress.com? :(,2
7,[deleted],1250145588,9a6xy,34,11,[deleted],Jazz,Just starting to listen to Jazz. Recommendati...,2
8,[deleted],1250176578,9ab7s,0,1,[deleted],Jazz,Jazz pioneer Rashied Ali Has died.,2
9,[deleted],1252880400,9k6pw,0,0,[deleted],Jazz,The Bad Plus - Cheney Pinata ... groundbreaki...,2
...,...,...,...,...,...,...,...,...,...
39951,zSyke99,1653782286,uzzbpg,1,1,[removed],electronicmusic,I made this playlist with my favorite Bitbird ...,20
39952,the_fire_tornado,1653795773,v02xjo,1,1,[removed],electronicmusic,Fire Tornado - Unknown,20
39953,the_fire_tornado,1653796729,v0360s,1,1,[removed],electronicmusic,Fire Tornado - Unknown (edm/dance),20
39954,Prestigious-Park9301,1653803513,v04saa,1,1,[removed],electronicmusic,"I'm looking for pop EDM songs that give you ""s...",20


**Conclusions:**
- The [deleted] and [removed] posts still have good material in their **title** column, so we should keep those documents.
- We will replace [deleted] and [removed] with empty strings.

In [79]:
# Replacing [deleted] and [removed]

df.loc[(df['selftext'] == '[deleted]') | (df['selftext'] == '[removed]'), 'selftext'] = ''

In [80]:
# double-check

df[(df['selftext'] == '[deleted]') | (df['selftext'] == '[removed]')]

# good - no [deleted] or [removed] left

,author,created_utc,id,num_comments,score,selftext,subreddit,title,period


In [81]:
# let's concatenate now

df['all_text'] = df['title'] + ' ' + df['selftext']

- Our new **all_text** column will be the text we work on from this point forward - we'll keep the original in case we need it later.
- It's time to perform the cleaning transforms!
- **NOTE**: This might be best as a consolidated function for production use, but I chose to keep it as separate steps to allow checking things as I go along.

In [82]:
# let's get rid of unicode/emojis
# https://stackoverflow.com/questions/44010727/remove-unicode-code-uxxx-in-string-python

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'[^\x00-\x7F]+', ' ', x))

In [83]:
# let's fix up the HTML entities - running this 2x because some of them are double-encoded

df['all_text'] = df['all_text'].map(lambda x: replace_entities(x))
df['all_text'] = df['all_text'].map(lambda x: replace_entities(x))

In [84]:
# bye bye literal escape characters

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'\n|\r|\t', ' ', x))

In [85]:
# no URLs please!

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'\bhttp(s?)://.+\b', ' ', x))

In [86]:
# Punctuation that isn't apostrophes

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'[.,/;:!?~@#$%^&*()_+=\[\]|<>"-]', ' ', x))

In [87]:
# Numbers (eek)

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'[0-9]', ' ', x))

In [88]:
# Double spaces

df['all_text'] = df['all_text'].map(lambda x: re.sub(r'\s+', ' ', x))

In [89]:
# Leading and trailing spaces

df['all_text'] = df['all_text'].map(lambda x: x.strip())

In [90]:
# How are we looking?

df['all_text']

0               Anyone know of any dark evil sounding jazz
1        In honor of Soil Pimp showing us how it's done...
2        anybody have that record of Charlie Christian ...
3        New Topline Image for Jazz subreddit Suggestio...
4              Teaching yourself Jazz Piano is it possible
                               ...                        
39951    I made this playlist with my favorite Bitbird ...
39952                                 Fire Tornado Unknown
39953                       Fire Tornado Unknown edm dance
39954    I'm looking for pop EDM songs that give you su...
39955            Enchanted Dark Fairy Forest Ambient Music
Name: all_text, Length: 39956, dtype: object

- We're in pretty good shape now in terms of clean **all_text**.
- If our cleaning resulted in any empty **all_text** fields, we should drop those, since they're now useless.
- As part of the survey for empty fields, we'll create an **all_text_len** field; we can also use this in EDA later.
- We'll also add an **all_text_word_len** field to use in EDA.
- We should also make sure the **subreddit** column is standardized to lowercase, just in case there are inconsistencies in capitalization coming back from the API - we don't want to generate new classes just because of a capital letter!

In [91]:
# let's get the character lengths - generally useful and also will let us know if we have any zeros now

df['all_text_len'] = df['all_text'].map(lambda x: len(str(x)))

In [92]:
# Just making sure - do we have empty all_texts?

df['all_text_len'].sort_values()

# yep we got zeros - they need to go

27671        0
25678        0
38465        0
38468        0
6307         0
         ...  
24705    15110
27618    16528
29366    16570
27225    17004
28106    17641
Name: all_text_len, Length: 39956, dtype: int64

In [93]:
# Let's see how much dropping the zero-length rows is gonna affect the respective classes

df[df['all_text_len'] == 0].groupby('subreddit').count()['all_text_len']

subreddit
Jazz                26
classicalmusic      13
electronicmusic    105
popheads            28
Name: all_text_len, dtype: int64

- Given that we have about 10,000 documents per genre, these losses seem acceptable.

In [94]:
df = df[df['all_text_len'] > 0]
df.shape

# looks good

(39784, 11)

In [95]:
# let's also make a length-in-words column

df['all_text_word_len'] = df['all_text'].map(lambda x: len(x.split()))

In [96]:
# we should fix up the subreddits so they're all lowercase

df['subreddit'] = df['subreddit'].str.lower()

In [97]:
# let's dump out the results so we can do some EDAing!

df.to_csv('../data/clean_posts.csv', index = False)

**FINAL NOTES**::
- The results of the retrieval are exported to (../data/clean_posts.csv).
- The next notebook in the series is [Exploratory Data Analysis](./03_eda.ipynb).